In [1]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
genres = ['blues', 'classical', 'country','disco','hiphop','jazz','metal','pop','reggae','rock']
frame_length = 0.025
frame_stride = 0.010

# 화음 판별 추출 array 생성
maj_template = np.array([1,0,0, 0,1,0, 0,1,0, 0,0,0])
min_template = np.array([1,0,0, 1,0,0, 0,1,0, 0,0,0])
N_template   = np.array([1,1,1, 1,1,1, 1,1,1, 1,1,1.]) / 4.
# Generate the weighting matrix that maps chroma to labels
weights = np.zeros((25, 12), dtype=float)
labels = ['C:maj', 'C#:maj', 'D:maj', 'D#:maj', 'E:maj', 'F:maj',
          'F#:maj', 'G:maj', 'G#:maj', 'A:maj', 'A#:maj', 'B:maj',
          'C:min', 'C#:min', 'D:min', 'D#:min', 'E:min', 'F:min',
          'F#:min', 'G:min', 'G#:min', 'A:min', 'A#:min', 'B:min',
          'N']
for c in range(12):
    weights[c, :] = np.roll(maj_template, c) # c:maj
    weights[c + 12, :] = np.roll(min_template, c)  # c:min
weights[-1] = N_template  # the last row is the no-chord class
# Make a self-loop transition matrix over 25 states
trans = librosa.sequence.transition_loop(25, 0.9)


song_name = list(np.full(1000,''))
tempos = np.full(1000,0)
chords_1 = list(np.full(1000,''))
chords_2 = list(np.full(1000,''))
for g in range(10):
    for i in range(100):
        ii = ''
        if i<10:
            ii = '0'+str(i)
        else:
            ii = str(i)
        y, sr = librosa.load(f'./genres/{genres[g]}/{genres[g]}.000{ii}.wav', sr=22050)
        
        song_name[g*100 + i] =f'{genres[g]}.000{ii}.wav'
        print(f'{genres[g]}.000{ii}.wav')
        
        # 음원의 bpm 추출
        tempo, beats = librosa.beat.beat_track(y=y, sr=sr)
        tempos[g*100 + i] = tempo
        try:
            # 음원의 화음 추출
            y = librosa.effects.harmonic(y, margin=4)
            chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
            # Map chroma (observations) to class (state) likelihoods
            probs = np.exp(weights.dot(chroma))  # P[class | chroma] ~= exp(template' chroma)
            probs /= probs.sum(axis=0, keepdims=True)  # probabilities must sum to 1 in each column
            # Compute independent frame-wise estimates
            chords_ind = np.argmax(probs, axis=0)
            # And viterbi estimates
            chords_vit = librosa.sequence.viterbi_discriminative(probs, trans)

            # 화음 추출
            series = pd.Series(chords_vit)
            dictionary = dict(series.value_counts())
            chord_rank = list(dictionary.keys())
            chord_rank_1 = chord_rank[0]
            chord_rank_2 = chord_rank[1]
            chords_1[g*100+i] = labels[chord_rank_1]
            chords_2[g*100+i] = labels[chord_rank_2]
            print(chords_1[i],chords_2[i])
        except:
            continue

blues.00000.wav
G:maj C:maj
blues.00001.wav
C:maj G:min
blues.00002.wav
E:min E:maj
blues.00003.wav
E:min E:maj
blues.00004.wav
A#:min A#:maj
blues.00005.wav
G:maj G:min
blues.00006.wav
F:maj F:min
blues.00007.wav
A#:maj A#:min
blues.00008.wav
E:min B:min
blues.00009.wav
G:maj G:min
blues.00010.wav
F:maj D#:maj
blues.00011.wav
E:min E:maj
blues.00012.wav
B:min E:maj
blues.00013.wav
B:min G#:min
blues.00014.wav
B:min A#:maj
blues.00015.wav
E:maj B:min
blues.00016.wav
A:maj C#:min
blues.00017.wav
A:min F#:min
blues.00018.wav
B:min D:maj
blues.00019.wav
A:min C#:min
blues.00020.wav
E:maj C:maj
blues.00021.wav
F#:min D:maj
blues.00022.wav
F#:min F#:maj
blues.00023.wav
A:maj A:min
blues.00024.wav
A#:min G#:min
blues.00025.wav
A#:maj E:maj
blues.00026.wav
E:maj A#:min
blues.00027.wav
E:maj B:min
blues.00028.wav
A#:maj A#:min
blues.00029.wav
D:min G:maj
blues.00030.wav
F:min C:min
blues.00031.wav
D:maj G:maj
blues.00032.wav
C:min F:maj
blues.00033.wav
F:maj D:min
blues.00034.wav
D:min D:maj
b

G:maj C:maj
country.00068.wav
F:maj F:min
country.00069.wav
G:maj C:maj
country.00070.wav
F:maj C:maj
country.00071.wav
F:maj C:min
country.00072.wav
G:maj C:maj
country.00073.wav
D#:maj G#:maj
country.00074.wav
D#:maj A#:min
country.00075.wav
C:maj F:maj
country.00076.wav
A#:maj F:maj
country.00077.wav
F:maj G:min
country.00078.wav
D#:maj F#:maj
country.00079.wav
D#:min D#:maj
country.00080.wav
C:maj G:maj
country.00081.wav
G:maj D:maj
country.00082.wav
A:maj F#:min
country.00083.wav
A#:min F:maj
country.00084.wav
C:min G:min
country.00085.wav
G:maj C:maj
country.00086.wav
E:min G:maj
country.00087.wav
F#:maj C#:maj
country.00088.wav
G:min F:maj
country.00089.wav
A:min C:maj
country.00090.wav
B:min A:maj
country.00091.wav
A:maj E:maj
country.00092.wav
E:maj F#:maj
country.00093.wav
E:maj B:maj
country.00094.wav
A:min F#:min
country.00095.wav
E:min A:maj
country.00096.wav
E:maj A:maj
country.00097.wav
F#:min B:min
country.00098.wav
C:maj F:maj
country.00099.wav
C:maj A:min
disco.00000.

G#:maj C#:maj
jazz.00054.wav
A:maj D:maj
jazz.00055.wav
C:maj G:min
jazz.00056.wav
E:min E:maj
jazz.00057.wav
B:maj E:min
jazz.00058.wav
G#:min C#:min
jazz.00059.wav
G:maj C:maj
jazz.00060.wav
A:maj A:min
jazz.00061.wav
G:min D:min
jazz.00062.wav
F:maj A#:maj
jazz.00063.wav
F:maj C:min
jazz.00064.wav
F:maj A#:maj
jazz.00065.wav
C:maj C:min
jazz.00066.wav
G:maj A:min
jazz.00067.wav
G:maj C:maj
jazz.00068.wav
F:maj F:min
jazz.00069.wav
G:maj C:maj
jazz.00070.wav
F:maj C:maj
jazz.00071.wav
F:maj C:min
jazz.00072.wav
G:maj C:maj
jazz.00073.wav
D#:maj G#:maj
jazz.00074.wav
D#:maj A#:min
jazz.00075.wav
C:maj F:maj
jazz.00076.wav
A#:maj F:maj
jazz.00077.wav
F:maj G:min
jazz.00078.wav
D#:maj F#:maj
jazz.00079.wav
D#:min D#:maj
jazz.00080.wav
C:maj G:maj
jazz.00081.wav
G:maj D:maj
jazz.00082.wav
A:maj F#:min
jazz.00083.wav
A#:min F:maj
jazz.00084.wav
C:min G:min
jazz.00085.wav
G:maj C:maj
jazz.00086.wav
E:min G:maj
jazz.00087.wav
F#:maj C#:maj
jazz.00088.wav
G:min F:maj
jazz.00089.wav
A:min C:m

C#:min G#:min
reggae.00048.wav
D#:maj G#:maj
reggae.00049.wav
G#:maj D#:min
reggae.00050.wav
B:maj E:min
reggae.00051.wav
E:maj A:maj
reggae.00052.wav
C:maj G:min
reggae.00053.wav
G#:maj C#:maj
reggae.00054.wav
A:maj D:maj
reggae.00055.wav
C:maj G:min
reggae.00056.wav
E:min E:maj
reggae.00057.wav
B:maj E:min
reggae.00058.wav
G#:min C#:min
reggae.00059.wav
G:maj C:maj
reggae.00060.wav
A:maj A:min
reggae.00061.wav
G:min D:min
reggae.00062.wav
F:maj A#:maj
reggae.00063.wav
F:maj C:min
reggae.00064.wav
F:maj A#:maj
reggae.00065.wav
C:maj C:min
reggae.00066.wav
G:maj A:min
reggae.00067.wav
G:maj C:maj
reggae.00068.wav
F:maj F:min
reggae.00069.wav
G:maj C:maj
reggae.00070.wav
reggae.00071.wav
F:maj C:min
reggae.00072.wav
G:maj C:maj
reggae.00073.wav
D#:maj G#:maj
reggae.00074.wav
D#:maj A#:min
reggae.00075.wav
C:maj F:maj
reggae.00076.wav
A#:maj F:maj
reggae.00077.wav
F:maj G:min
reggae.00078.wav
D#:maj F#:maj
reggae.00079.wav
D#:min D#:maj
reggae.00080.wav
C:maj G:maj
reggae.00081.wav
G:maj

In [26]:
music_data = pd.DataFrame(np.zeros((1000,4)),columns = ['wav_name','tempos','chord_rank_1','chord_rank_2'])
music_data['wav_name'] = song_name
music_data['tempos'] = tempos
music_data['chord_rank_1'] = chords_1
music_data['chord_rank_2'] = chords_2

In [27]:
music_data

,wav_name,tempos,chord_rank_1,chord_rank_2
0,blues.00000.wav,123,G:maj,C:maj
1,blues.00001.wav,67,C:maj,G:min
2,blues.00002.wav,161,E:min,E:maj
3,blues.00003.wav,63,E:min,E:maj
4,blues.00004.wav,135,A#:min,A#:maj
...,...,...,...,...
995,rock.00095.wav,103,D:maj,G:maj
996,rock.00096.wav,117,A:maj,C#:min
997,rock.00097.wav,129,D:maj,G:maj
998,rock.00098.wav,73,C:maj,F:maj
